In [1]:
import requests

from datetime import datetime

import pandas as pd


In [2]:

API_KEY = "2lzP1WX4IAnQT29AfbjhrHbaPB2qIR2IUWmRkY4x"
url = f"https://api.nasa.gov/neo/rest/v1/feed?start_date=2024-01-01&end_date=2024-01-08&api_key={API_KEY}"

asteroids_data = []  # Empty list to store asteroid information
target = 10000      # Target number of asteroids to fetch

while len(asteroids_data) < target:  # Loop until we reach the target count
    url_response = requests.get(url)  # Fetch data from NASA API
    data = url_response.json()        # Convert the response to JSON
    details = data['near_earth_objects']  # Extract near-earth objects

    for date, asteroids in details.items():  # Loop through each date and its asteroids
        for ast in asteroids:  # Loop through each asteroid
            asteroid_info = {  # Store asteroid details in a dictionary
                "Id": int(ast['id']),
                "Name": str(ast['name']),
                "Absolute_magnitude_h": float(ast['absolute_magnitude_h']),
                "Estimated_dia_min_km": float(ast['estimated_diameter']['kilometers']['estimated_diameter_min']),
                "Estimated_dia_max_km": float(ast['estimated_diameter']['kilometers']['estimated_diameter_max']),
                "Is_potentially_hazardous_asteroid": ast['is_potentially_hazardous_asteroid'],
                "Neo_reference_id": int(ast['neo_reference_id']),
                "Close_approach_date": datetime.strptime(ast['close_approach_data'][0]['close_approach_date'], "%Y-%m-%d"),
                "Relative_velocity_kmph": float(ast['close_approach_data'][0]['relative_velocity']['kilometers_per_hour']),
                "Astronomical": float(ast['close_approach_data'][0]['miss_distance']['astronomical']),
                "Miss_distance_km": float(ast['close_approach_data'][0]['miss_distance']['kilometers']),
                "Miss_distance_lunar": float(ast['close_approach_data'][0]['miss_distance']['lunar']),
                "Orbiting_body": str(ast['close_approach_data'][0]['orbiting_body'])
            }

            asteroids_data.append(asteroid_info)  # Append asteroid info to the list

            # Break if we reached the target
        if len(asteroids_data) >= target:
                break

    if len(asteroids_data) >= target:  # Break the outer loop if target reached
            break

    url = data['links']['next']  # Move to the next page of results



In [3]:
len(asteroids_data)

10000

In [4]:
asteroids_data[0]

{'Id': 2415949,
 'Name': '415949 (2001 XY10)',
 'Absolute_magnitude_h': 19.37,
 'Estimated_dia_min_km': 0.3552670883,
 'Estimated_dia_max_km': 0.7944013596,
 'Is_potentially_hazardous_asteroid': False,
 'Neo_reference_id': 2415949,
 'Close_approach_date': datetime.datetime(2024, 1, 2, 0, 0),
 'Relative_velocity_kmph': 57205.8951204341,
 'Astronomical': 0.3372535274,
 'Miss_distance_km': 50452409.349026635,
 'Miss_distance_lunar': 131.1916221586,
 'Orbiting_body': 'Earth'}

In [5]:
# MYSQL connection establish

import mysql.connector as db

conn = db.connect(
    host='localhost',
    user='vignesh',         
    password='1234', 
    database='guvi_neo_project' 
)
cursor = conn.cursor()

cursor.execute("USE guvi_neo_project;") 

In [6]:
# create two tables

cursor.execute("""
CREATE TABLE asteroids (
    id INT,
    name VARCHAR(255),
    absolute_magnitude_h FLOAT,
    estimated_diameter_min_km FLOAT,
    estimated_diameter_max_km FLOAT,
    is_potentially_hazardous_asteroid BOOLEAN
);
""")

cursor.execute("""
CREATE TABLE close_approach (
    neo_reference_id INT,
    close_approach_date DATE,
    relative_velocity_kmph FLOAT,
    astronomical FLOAT,
    miss_distance_km FLOAT,
    miss_distance_lunar FLOAT,
    orbiting_body VARCHAR(255)
);
""")

In [7]:
#insert the value in table

asteroids_values = [
    (
        ast['Id'],
        ast['Name'],
        ast['Absolute_magnitude_h'],
        ast['Estimated_dia_min_km'],
        ast['Estimated_dia_max_km'],
        ast['Is_potentially_hazardous_asteroid']
    )
    for ast in asteroids_data
]

close_approach_values = [
    (
        ast['Neo_reference_id'],
        ast['Close_approach_date'].date(),  # convert datetime to date
        ast['Relative_velocity_kmph'],
        ast['Astronomical'],
        ast['Miss_distance_km'],
        ast['Miss_distance_lunar'],
        ast['Orbiting_body']
    )
    for ast in asteroids_data
]

cursor.executemany("""
    INSERT INTO asteroids (Id, Name, Absolute_magnitude_h, Estimated_diameter_min_km, Estimated_diameter_max_km, Is_potentially_hazardous_asteroid)
    VALUES (%s, %s, %s, %s, %s, %s)
""", asteroids_values)

cursor.executemany("""
    INSERT INTO close_approach (Neo_reference_id, Close_approach_date, Relative_velocity_kmph, Astronomical, Miss_distance_km, Miss_distance_lunar, Orbiting_body)
    VALUES (%s, %s, %s, %s, %s, %s, %s)
""", close_approach_values)

conn.commit()

